<a href="https://colab.research.google.com/github/ikram2500/sentimentanalysis/blob/main/Sentiment140_dataset_with_1_6_million_tweets.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
! pip install kaggle

In [ ]:
# prompt:  Configure the path of kaggle.json file

!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 /root/.kaggle/kaggle.json


In [ ]:
!kaggle datasets download --force -d kazanova/sentiment140

 83% 67.0M/80.9M [00:00<00:00, 124MB/s]
100% 80.9M/80.9M [00:00<00:00, 122MB/s]


In [ ]:
#extracting the compressed file
from zipfile import ZipFile
dataset = "/content/sentiment140.zip"

with ZipFile(dataset, 'r') as zip:
  zip.extractall()
  print('The dataset is extracted')

The dataset is extracted


In [ ]:
!pip install

ERROR: You must give at least one requirement to install (see "pip help install")


In [ ]:
import pandas as pd
import numpy as np
import re
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [ ]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
# printing the stopwords
print(len(stopwords.words('english')))

179


**Data Preprocessing**

In [ ]:
twitter_data = pd.read_csv('/content/training.1600000.processed.noemoticon.csv', encoding='ISO-8859-1')

In [ ]:
twitter_data.shape

(1599999, 6)

In [ ]:
twitter_data.head()

,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, that's a bummer. You shoulda got David Carr of Third Day to do it. ;D"
0,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
1,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
2,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
3,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."
4,0,1467811372,Mon Apr 06 22:20:00 PDT 2009,NO_QUERY,joy_wolf,@Kwesidei not the whole crew


**Column names**

In [ ]:
# naming the columns and reading the dataset again
column_names = ['target', 'id', 'date','flag', 'user','text' ]
twitter_data = pd.read_csv('/content/training.1600000.processed.noemoticon.csv', encoding='ISO-8859-1', names=column_names)

In [ ]:
twitter_data.head()

,target,id,date,flag,user,text
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."


In [ ]:
twitter_data.shape

(1600000, 6)

In [ ]:
twitter_data.isnull().sum()

target    0
id        0
date      0
flag      0
user      0
text      0
dtype: int64

In [ ]:
twitter_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1600000 entries, 0 to 1599999
Data columns (total 6 columns):
 #   Column  Non-Null Count    Dtype 
---  ------  --------------    ----- 
 0   target  1600000 non-null  int64 
 1   id      1600000 non-null  int64 
 2   date    1600000 non-null  object
 3   flag    1600000 non-null  object
 4   user    1600000 non-null  object
 5   text    1600000 non-null  object
dtypes: int64(2), object(4)
memory usage: 73.2+ MB


In [ ]:
twitter_data['target'].value_counts()

target
0    800000
4    800000
Name: count, dtype: int64

**Convert the target label 4 to 1**

In [ ]:
twitter_data.replace({'target':{4:1}}, inplace=True)

In [ ]:
twitter_data['target'].value_counts()

target
0    800000
1    800000
Name: count, dtype: int64

In [ ]:
# 0 means negative tweet and 1 means positive


In [ ]:
twitter_data = twitter_data[['target', 'text']]
twitter_data.head()

,target,text
0,0,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,is upset that he can't update his Facebook by ...
2,0,@Kenichan I dived many times for the ball. Man...
3,0,my whole body feels itchy and like its on fire
4,0,"@nationwideclass no, it's not behaving at all...."


**STEMMING**

In [ ]:
#stemming is the process of reducing a word to its Root word
port_stem = PorterStemmer()

In [ ]:
from nltk.stem import SnowballStemmer
from nltk.corpus import stopwords
import re

to_remove = "@\S+|https?:\S+|http?:\S|[^A-Za-z0-9]+"
stopwords_ = stopwords.words('english')
stemmer = SnowballStemmer("english")

def preprocess_text(text, stem=False):
    # Remove link,user and special characters
    text = re.sub(to_remove, ' ', text.lower()).strip()
    tokens = text.split()
    # Remove stop words
    tokens = [token for token in tokens if token not in stopwords_]
    # Stemming the tokens
    if stem:
        tokens = [stemmer.stem(token) for token in tokens]
    return " ".join(tokens)

In [ ]:
preprocess_text(twitter_data.text[10])

'spring break plain city snowing'

In [ ]:
preprocess_text(twitter_data.text[10], stem=True)

'spring break plain citi snow'

In [ ]:
%%time
twitter_data['stemmed_content'] = twitter_data.text.apply(lambda x: preprocess_text(x, stem=True))

CPU times: user 4min 2s, sys: 858 ms, total: 4min 3s
Wall time: 4min 15s


In [ ]:
"""def stemming(content):
  stemmed_content = re.sub(  '[^a-zA-Z]'   ,   ' '    , content )
  stemmed_content = stemmed_content.lower()
  stemmed_content = stemmed_content.split()
  stemmed_content = [port_stem.stem(word) for word in stemmed_content if word in stopwords.words('english')]
  stemmed_content = ' '.join(stemmed_content)

  return stemmed_content"""

"def stemming(content):\n  stemmed_content = re.sub(  '[^a-zA-Z]'   ,   ' '    , content )\n  stemmed_content = stemmed_content.lower()\n  stemmed_content = stemmed_content.split()\n  stemmed_content = [port_stem.stem(word) for word in stemmed_content if word in stopwords.words('english')]\n  stemmed_content = ' '.join(stemmed_content)\n\n  return stemmed_content"

In [ ]:
"""twitter_data['stemmed_content'] = twitter_data['text'].apply(stemming)"""

"twitter_data['stemmed_content'] = twitter_data['text'].apply(stemming)"

In [ ]:
twitter_data.head()

,target,text,stemmed_content
0,0,"@switchfoot http://twitpic.com/2y1zl - Awww, t...",awww bummer shoulda got david carr third day
1,0,is upset that he can't update his Facebook by ...,upset updat facebook text might cri result sch...
2,0,@Kenichan I dived many times for the ball. Man...,dive mani time ball manag save 50 rest go bound
3,0,my whole body feels itchy and like its on fire,whole bodi feel itchi like fire
4,0,"@nationwideclass no, it's not behaving at all....",behav mad see


In [ ]:
print(twitter_data['stemmed_content'])

0               awww bummer shoulda got david carr third day
1          upset updat facebook text might cri result sch...
2            dive mani time ball manag save 50 rest go bound
3                            whole bodi feel itchi like fire
4                                              behav mad see
                                 ...                        
1599995                           woke school best feel ever
1599996              thewdb com cool hear old walt interview
1599997                         readi mojo makeov ask detail
1599998    happi 38th birthday boo alll time tupac amaru ...
1599999    happi charitytuesday thenspcc sparkschar speak...
Name: stemmed_content, Length: 1600000, dtype: object


In [ ]:
"""# separating the data and label
X = twitter_data['stemmed_content'].values
Y = twitter_data['target'].values"""

"# separating the data and label\nX = twitter_data['stemmed_content'].values\nY = twitter_data['target'].values"

In [ ]:
X = twitter_data['stemmed_content']
y = twitter_data['target']

In [ ]:
print(X)

0               awww bummer shoulda got david carr third day
1          upset updat facebook text might cri result sch...
2            dive mani time ball manag save 50 rest go bound
3                            whole bodi feel itchi like fire
4                                              behav mad see
                                 ...                        
1599995                           woke school best feel ever
1599996              thewdb com cool hear old walt interview
1599997                         readi mojo makeov ask detail
1599998    happi 38th birthday boo alll time tupac amaru ...
1599999    happi charitytuesday thenspcc sparkschar speak...
Name: stemmed_content, Length: 1600000, dtype: object


In [ ]:
print(y)

0          0
1          0
2          0
3          0
4          0
          ..
1599995    1
1599996    1
1599997    1
1599998    1
1599999    1
Name: target, Length: 1600000, dtype: int64


In [ ]:
#splitting the data into training and testing data
"""X_train, y_train, X_test, Y_test = train_test_split(X, y, test_size=0.2,  random_state=2)"""
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

(1280000,) (320000,) (1280000,) (320000,)


In [ ]:
print(X_train.shape, X_test.shape)

(1280000,) (320000,)


In [ ]:
X_train.head()

1374558    ya quot like palm pre touchston charger readyn...
1389115                felt earthquak afternoon seem epicent
1137831                               ruffl shirt like likey
790714     pretti bad night crappi morn fml buttfac didnt...
1117911                                      yeah clear view
Name: stemmed_content, dtype: object

**Converting text data to numerical data**

In [ ]:
"""vectorizer = TfidfVectorizer()

X_train = vectorizer.fit_transform(X_train)
X_test = vectorizer.fit_transform(X_test)"""

'vectorizer = TfidfVectorizer()\n\nX_train = vectorizer.fit_transform(X_train)\nX_test = vectorizer.fit_transform(X_test)'

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer()
X_train = vectorizer.fit_transform(X_train)
X_test = vectorizer.fit_transform(X_test)

In [ ]:
print(X_train)

  (0, 173397)	0.39496863868575965
  (0, 179450)	0.18286793094562143
  (0, 27470)	0.23433369622263847
  (0, 89130)	0.12379726829124904
  (0, 202344)	0.18576604457914483
  (0, 245359)	0.17492278344726647
  (0, 179490)	0.44394329041768577
  (0, 43851)	0.28272339868646995
  (0, 221649)	0.421906384936126
  (0, 173165)	0.25619687157650967
  (0, 164431)	0.27974165542308865
  (0, 128722)	0.1268818454298196
  (0, 177268)	0.15319316335344269
  (0, 243930)	0.1916546484518264
  (1, 71560)	0.6460764147003921
  (1, 191735)	0.31102688787535854
  (1, 11365)	0.3482234914331628
  (1, 67596)	0.46964646292142814
  (1, 77108)	0.3794998567403495
  (2, 128773)	0.5692765126035431
  (2, 195022)	0.413870419463041
  (2, 186099)	0.673663012994704
  (2, 128722)	0.22541888344664407
  (3, 220103)	0.17587827618402033
  (3, 241317)	0.1668011845416974
  :	:
  (1279997, 4081)	0.7649609836209437
  (1279997, 22083)	0.5112145533339348
  (1279997, 178859)	0.3917835805610845
  (1279998, 187329)	0.39716622071537633
  (1279998

In [ ]:
print(X_test)

  (0, 63976)	0.5404433014717855
  (0, 41039)	0.4412017919159379
  (0, 4910)	0.7164230710302694
  (1, 71888)	0.7463521654096579
  (1, 7321)	0.4567505572270011
  (1, 90345)	0.3306597869713125
  (1, 20781)	0.35355548212582527
  (2, 16101)	0.2218458156946857
  (2, 42759)	0.2524875050875232
  (2, 95596)	0.29805460471718714
  (2, 79739)	0.36836456938790435
  (2, 51641)	0.15658618720443362
  (2, 37765)	0.21266560385648964
  (2, 48526)	0.3470601896448233
  (2, 38525)	0.25729846229592046
  (2, 87885)	0.16520408236928832
  (2, 61645)	0.2137351064776634
  (2, 39953)	0.21469413066898077
  (2, 50158)	0.29906306229926627
  (2, 26435)	0.32622761556049873
  (2, 30677)	0.2474494080635453
  (2, 49024)	0.17304128751156758
  (3, 83373)	0.31669095980232687
  (3, 73289)	0.34561505486219835
  (3, 83328)	0.29602863970637144
  :	:
  (319994, 58453)	0.23269397137772063
  (319994, 17949)	0.19649719332206966
  (319994, 71133)	0.15815164282475952
  (319994, 20109)	0.15508782423901443
  (319995, 16588)	0.7794494501

**Training the ML model**

In [ ]:
model = LogisticRegression(max_iter= 1000)

In [ ]:
model.fit(X_train, y_train)

LogisticRegression(max_iter=1000)

**Model Evaluetion**

***Accuracy score**

In [ ]:
X_train_prediction = model.predict(X_train)
training_data_accuracy = accuracy_score(y_train, X_train_prediction)

In [ ]:
print('training data accuracy score: ', training_data_accuracy)

training data accuracy score:  0.79485625


In [ ]:
X_train_prediction = model.predict(X_train)
training_data_accuracy = accuracy_score(y_train, X_train_prediction)

In [ ]:
print('test data accuracy score: ', training_data_accuracy)

test data accuracy score:  0.79485625


##Saving the train Model

In [ ]:
import pickle

In [ ]:
filename = 'trained_model.sav'
pickle.dump(model, open(filename, 'wb'))

### Using the saved model for future prediction

In [ ]:
loaded_model = pickle.load(open('/content/trained_model.sav', 'rb'))